In [12]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64


from crud import CRUD



###########################
# Data Manipulation / Model
###########################
port = "51915"
database = "AAC"
collection = "animals"
username = "aacuser"
password = "8rBn8A8CZ"
shelter = CRUD(port, database, collection, username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({'animal_type':'Dog'}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div([
        html.Div([
            html.Div([
                html.A([
                    html.Img(
                        src='data:image/png;base64,{}'.format(encoded_image.decode()),
                        style = {'height':'100px', 'width':'100px'})],
                    href = 'https://www.snhu.edu/')],
                style = {'float': 'center', 'display':'inline-block'})],   
            style = {'width':'100px', 'display': 'inline-block', 'overflow':'auto'}),
        
        html.Div([
            html.Div([
                html.B(html.H1('SNHU CS-340 Dashboard'))],
                style = {'float': 'center', 'display': 'inline-block'})],
            style = {'width':'calc(100%-100px)', 'display': 'inline-block', 'overflow':'auto'}
        )],     
        style = {'width':'100%', 'display':'block', 'overflow':'auto'}
            
            
    ),
    html.Footer([html.H5(children = "Created by Christopher George 2021")],
    
        style = {'background-color':'#45433d', 'color':'white', 'height':'15px'}
    ),
    html.Hr(),
    html.Div(
        
        dcc.RadioItems(
        options = [
            {'label':"Water Rescue", 'value':'1'},
            {'label':"Wilderness Rescue", 'value':'2'},
            {'label':"Individual Tracking", 'value':'3'},
            {'label':"reset", 'value':'0'},
        ], 
        id = 'filter-type')
    ),
    html.Hr(),
    dt.DataTable(id='datatable-id', style_table = {'height':'300px', 'width':'100vw', 'overflow':'auto'}),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
html.Footer([html.H5(children = "Created by Christopher George 2021")],
    
    style = {'background-color':'#45433d', 'color':'white', 'height':'15px'}
)
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    if(filter_type == '1'):
        df = pd.DataFrame.from_records(shelter.read({
            'animal_type':'Dog',
            'breed':{'$in':[
                'Labrador Retriever Mix',
                'Chesapeake Bay Retriever',
                'Newfoundland']},
            'sex_upon_outcome':'Intact Female',
            'age_upon_outcome_in_weeks':{'$gt':26,'$lt':156}
            }))
    elif(filter_type == '2'):
        df = pd.DataFrame.from_records(shelter.read({
            'animal_type':'Dog',
            'breed':{'$in':['German Shepherd',
                            'Alaskan Malamute',
                            'Old English Sheepdog',
                            'Siberian Husky',
                            'Rottweiler']},
            'sex_upon_outcome':'Intact Male',
            'age_upon_outcome_in_weeks':{'$gt':26,'$lt':156}
            }))
    elif(filter_type == '3'):
        df = pd.DataFrame.from_records(shelter.read({
            'animal_type':'Dog',
            'breed':{'$in':['Doberman Pinscher',
                            'GermanShepherd',
                            'Golden Retriever',
                            'Bloodhound',
                            'Rottweiler']},
            'sex_upon_outcome':'Intact Male',
            'age_upon_outcome_in_weeks':{'$gt':20,'$lt':300}
            }))
    else:
        df = pd.DataFrame.from_records(shelter.read({'animal_type':'Dog'}))    
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    data = viewData

    for i in data:
        i.update({'count': 1})
        
    return [
        dcc.Graph(            
            figure = px.pie(data, values = "count", names = "breed" )
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
    data = viewData
    markerList = [dl.TileLayer(id="base-layer-id")]
    x = 0
    for i in data:
        markerList.append(
        dl.Marker(position=[i.get('location_lat'), i.get('location_long')], children=[
                dl.Tooltip(i.get('breed')),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(i.get('name'))
                ])
            ])
        )
        if x == 0:
            center_cords = [i.get('location_lat'), i.get('location_long')]
            x = 1

        
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
        center=center_cords,
        zoom=9, 
        children=markerList
        )
    ]

app
